In [1]:
import numpy as np 
import pandas as pd
import json 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

from tqdm import tqdm
tqdm.pandas()
from sklearn.linear_model import ElasticNet
import statsmodels.api as sm
from copy import deepcopy

import Utils
import models

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel('PMT_march24.xlsx', sheet_name='teens')

In [3]:
df

,age,code,ACC_ADD1,ACC_ADD2,ACC_ADD3,ACC_DIV1,ACC_DIV2,ACC_DIV3,ACC_MUL1,ACC_MUL2,...,N_ADD3,N_DIV1,N_DIV2,N_DIV3,N_MUL1,N_MUL2,N_MUL3,N_SUB1,N_SUB2,N_SUB3
0,15,P-AYU-15-F-9R-ORLO-MOON2021-IDXX-2292,1.000000,0.894737,0.900000,0.840000,0.444444,0.181818,1.000000,0.666667,...,9.0,21.0,4.0,2.0,29.0,6.0,8.0,27.0,9.0,4.0
1,13,P-DKO-13-M-7E-1793-MOON2021-SGXX-2303,0.900000,0.916667,0.714286,0.816327,0.714286,0.625000,0.963636,0.764706,...,10.0,40.0,10.0,5.0,53.0,13.0,6.0,50.0,15.0,5.0
2,13,P-AAD-13-M-7C-1874-MOON2021-SGXX-2323,0.878788,0.750000,0.375000,0.782609,0.500000,0.125000,0.916667,0.350000,...,3.0,18.0,2.0,1.0,22.0,7.0,6.0,22.0,7.0,4.0
3,10,P-ABO-10-F-5I-1514-MOON2021-AFXX-2370,1.000000,0.500000,0.100000,0.789474,0.263158,0.117647,0.800000,0.222222,...,1.0,15.0,5.0,2.0,12.0,2.0,4.0,9.0,5.0,2.0
4,15,P-ABY-15-F-9G-RELI-IZON2021-AIXX-2325,0.914286,0.900000,0.714286,0.815789,0.769231,0.600000,0.921053,0.384615,...,10.0,31.0,10.0,6.0,35.0,5.0,6.0,39.0,10.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,12,P-YPA-12-F-6E-1205-MOON2021-ADXX-2258,0.944444,1.000000,0.750000,0.956522,0.666667,0.666667,0.950000,0.800000,...,3.0,22.0,4.0,2.0,19.0,4.0,0.0,20.0,3.0,5.0
292,17,P-YPO-17-F-BF1-HSEL-MOON2021-APXX-2306,0.936170,0.826087,0.461538,0.944444,0.750000,0.444444,1.000000,0.722222,...,6.0,51.0,9.0,4.0,58.0,13.0,7.0,49.0,15.0,7.0
293,15,P-YVE-15-F-9A-0029-IZON2021-SBXX-2260,0.961538,0.863636,0.631579,0.984375,0.533333,0.800000,0.986111,0.900000,...,12.0,63.0,8.0,8.0,71.0,18.0,4.0,63.0,17.0,9.0
294,14,P-ZBR-14-F-8B-1164-MOON2021-ADXX-2312,0.954545,0.714286,0.666667,0.900000,0.666667,0.857143,0.937500,0.571429,...,6.0,27.0,6.0,6.0,30.0,4.0,2.0,27.0,6.0,3.0


In [4]:
df.columns

Index(['age', 'code', 'ACC_ADD1', 'ACC_ADD2', 'ACC_ADD3', 'ACC_DIV1',
       'ACC_DIV2', 'ACC_DIV3', 'ACC_MUL1', 'ACC_MUL2', 'ACC_MUL3', 'ACC_SUB1',
       'ACC_SUB2', 'ACC_SUB3', 'RT_ADD1', 'RT_ADD2', 'RT_ADD3', 'RT_DIV1',
       'RT_DIV2', 'RT_DIV3', 'RT_MUL1', 'RT_MUL2', 'RT_MUL3', 'RT_SUB1',
       'RT_SUB2', 'RT_SUB3', 'N_ADD1', 'N_ADD2', 'N_ADD3', 'N_DIV1', 'N_DIV2',
       'N_DIV3', 'N_MUL1', 'N_MUL2', 'N_MUL3', 'N_SUB1', 'N_SUB2', 'N_SUB3'],
      dtype='object')

In [5]:
df = df.dropna()

df = df.drop(columns=['code', ])

seed = 0xAB0BA
np.random.seed(seed)

In [6]:
df['Accuracy ALL level1'] = df['ACC_ADD1'] + df['ACC_SUB1'] + df['ACC_MUL1'] + df['ACC_DIV1']
df['Accuracy ALL level2'] = df['ACC_ADD2'] + df['ACC_SUB2'] + df['ACC_MUL2'] + df['ACC_DIV2']
df['Accuracy ALL level3'] = df['ACC_ADD3'] + df['ACC_SUB3'] + df['ACC_MUL3'] + df['ACC_DIV3']
df['Accuracy ALL ADD'] = df['ACC_ADD1'] + df['ACC_ADD2'] + df['ACC_ADD3']
df['Accuracy ALL SUB'] = df['ACC_SUB1'] + df['ACC_SUB2'] + df['ACC_SUB3']
df['Accuracy ALL MUL'] = df['ACC_MUL1'] + df['ACC_MUL2'] + df['ACC_MUL3']
df['Accuracy ALL DIV'] = df['ACC_DIV1'] + df['ACC_DIV2'] + df['ACC_DIV3']


In [7]:
df

,age,ACC_ADD1,ACC_ADD2,ACC_ADD3,ACC_DIV1,ACC_DIV2,ACC_DIV3,ACC_MUL1,ACC_MUL2,ACC_MUL3,...,N_SUB1,N_SUB2,N_SUB3,Accuracy ALL level1,Accuracy ALL level2,Accuracy ALL level3,Accuracy ALL ADD,Accuracy ALL SUB,Accuracy ALL MUL,Accuracy ALL DIV
0,15,1.000000,0.894737,0.900000,0.840000,0.444444,0.181818,1.000000,0.666667,0.347826,...,27.0,9.0,4.0,3.840000,2.568348,1.762978,2.794737,1.895833,2.014493,1.466263
1,13,0.900000,0.916667,0.714286,0.816327,0.714286,0.625000,0.963636,0.764706,0.500000,...,50.0,15.0,5.0,3.623359,3.145658,2.255952,2.530952,2.110063,2.228342,2.155612
2,13,0.878788,0.750000,0.375000,0.782609,0.500000,0.125000,0.916667,0.350000,0.461538,...,22.0,7.0,4.0,3.424217,2.236364,1.294872,2.003788,1.815851,1.728205,1.407609
3,10,1.000000,0.500000,0.100000,0.789474,0.263158,0.117647,0.800000,0.222222,0.235294,...,9.0,5.0,2.0,3.339474,1.318713,0.606787,1.600000,1.237179,1.257516,1.170279
4,15,0.914286,0.900000,0.714286,0.815789,0.769231,0.600000,0.921053,0.384615,0.545455,...,39.0,10.0,4.0,3.537491,2.642081,2.193074,2.528571,1.807932,1.851123,2.185020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,12,0.944444,1.000000,0.750000,0.956522,0.666667,0.666667,0.950000,0.800000,0.000000,...,20.0,3.0,5.0,3.850966,3.466667,2.250000,2.694444,2.833333,1.750000,2.289855
292,17,0.936170,0.826087,0.461538,0.944444,0.750000,0.444444,1.000000,0.722222,0.636364,...,49.0,15.0,7.0,3.860615,3.131643,2.178710,2.223796,2.449697,2.358586,2.138889
293,15,0.961538,0.863636,0.631579,0.984375,0.533333,0.800000,0.986111,0.900000,0.307692,...,63.0,17.0,9.0,3.916400,3.069697,2.339271,2.456754,2.357102,2.193803,2.317708
294,14,0.954545,0.714286,0.666667,0.900000,0.666667,0.857143,0.937500,0.571429,0.500000,...,27.0,6.0,3.0,3.692045,2.552381,2.623810,2.335498,2.100000,2.008929,2.423810


In [8]:
df.columns

Index(['age', 'ACC_ADD1', 'ACC_ADD2', 'ACC_ADD3', 'ACC_DIV1', 'ACC_DIV2',
       'ACC_DIV3', 'ACC_MUL1', 'ACC_MUL2', 'ACC_MUL3', 'ACC_SUB1', 'ACC_SUB2',
       'ACC_SUB3', 'RT_ADD1', 'RT_ADD2', 'RT_ADD3', 'RT_DIV1', 'RT_DIV2',
       'RT_DIV3', 'RT_MUL1', 'RT_MUL2', 'RT_MUL3', 'RT_SUB1', 'RT_SUB2',
       'RT_SUB3', 'N_ADD1', 'N_ADD2', 'N_ADD3', 'N_DIV1', 'N_DIV2', 'N_DIV3',
       'N_MUL1', 'N_MUL2', 'N_MUL3', 'N_SUB1', 'N_SUB2', 'N_SUB3',
       'Accuracy ALL level1', 'Accuracy ALL level2', 'Accuracy ALL level3',
       'Accuracy ALL ADD', 'Accuracy ALL SUB', 'Accuracy ALL MUL',
       'Accuracy ALL DIV'],
      dtype='object')

In [9]:
df['RT ALL level1'] = df['RT_ADD1'] + df['RT_SUB1'] + df['RT_MUL1'] + df['RT_DIV1']
df['RT ALL level2'] = df['RT_ADD2'] + df['RT_SUB2'] + df['RT_MUL2'] + df['RT_DIV2']
df['RT ALL level3'] = df['RT_ADD3'] + df['RT_SUB3'] + df['RT_MUL3'] + df['RT_DIV3']
df['RT ALL ADD'] = df['RT_ADD1'] + df['RT_ADD2'] + df['RT_ADD3']
df['RT ALL SUB'] = df['RT_SUB1'] + df['RT_SUB2'] + df['RT_SUB3']
df['RT ALL MUL'] = df['RT_MUL1'] + df['RT_MUL2'] + df['RT_MUL3']
df['RT ALL DIV'] = df['RT_DIV1'] + df['RT_DIV2'] + df['RT_DIV3']


In [10]:
df

,age,ACC_ADD1,ACC_ADD2,ACC_ADD3,ACC_DIV1,ACC_DIV2,ACC_DIV3,ACC_MUL1,ACC_MUL2,ACC_MUL3,...,Accuracy ALL SUB,Accuracy ALL MUL,Accuracy ALL DIV,RT ALL level1,RT ALL level2,RT ALL level3,RT ALL ADD,RT ALL SUB,RT ALL MUL,RT ALL DIV
0,15,1.000000,0.894737,0.900000,0.840000,0.444444,0.181818,1.000000,0.666667,0.347826,...,1.895833,2.014493,1.466263,12.784671,26.872833,24.886377,15.885332,14.313627,15.170376,19.174545
1,13,0.900000,0.916667,0.714286,0.816327,0.714286,0.625000,0.963636,0.764706,0.500000,...,2.110063,2.228342,2.155612,6.715896,19.257381,30.756590,11.301028,13.281255,13.907643,18.239942
2,13,0.878788,0.750000,0.375000,0.782609,0.500000,0.125000,0.916667,0.350000,0.461538,...,1.815851,1.728205,1.407609,13.095643,29.707858,31.855985,18.547876,18.967643,13.257543,23.886424
3,10,1.000000,0.500000,0.100000,0.789474,0.263158,0.117647,0.800000,0.222222,0.235294,...,1.237179,1.257516,1.170279,18.950305,29.181643,24.489320,20.642950,17.828351,19.639605,14.510362
4,15,0.914286,0.900000,0.714286,0.815789,0.769231,0.600000,0.921053,0.384615,0.545455,...,1.807932,1.851123,2.185020,8.439596,21.818523,29.338398,11.856857,14.896197,15.596813,17.246651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,12,0.944444,1.000000,0.750000,0.956522,0.666667,0.666667,0.950000,0.800000,0.000000,...,2.833333,1.750000,2.289855,15.635999,52.633191,57.213584,28.645941,30.074701,36.969901,29.792232
292,17,0.936170,0.826087,0.461538,0.944444,0.750000,0.444444,1.000000,0.722222,0.636364,...,2.449697,2.358586,2.138889,6.347675,20.892945,30.621481,11.693794,14.240663,13.994284,17.933361
293,15,0.961538,0.863636,0.631579,0.984375,0.533333,0.800000,0.986111,0.900000,0.307692,...,2.357102,2.193803,2.317708,5.159896,18.475268,24.366673,9.974687,11.292898,11.748761,14.985491
294,14,0.954545,0.714286,0.666667,0.900000,0.666667,0.857143,0.937500,0.571429,0.500000,...,2.100000,2.008929,2.423810,10.249226,34.998627,49.874479,16.202422,26.603100,30.166192,22.150617


In [11]:
# Calculating new columns for aggregated NofCorr
df['NofCorr ALL level1'] = df['N_ADD1'] + df['N_SUB1'] + df['N_MUL1'] + df['N_DIV1']
df['NofCorr ALL level2'] = df['N_ADD2'] + df['N_SUB2'] + df['N_MUL2'] + df['N_DIV2']
df['NofCorr ALL level3'] = df['N_ADD3'] + df['N_SUB3'] + df['N_MUL3'] + df['N_DIV3']
df['NofCorr ALL ADD'] = df['N_ADD1'] + df['N_ADD2'] + df['N_ADD3']
df['NofCorr ALL SUB'] = df['N_SUB1'] + df['N_SUB2'] + df['N_SUB3']
df['NofCorr ALL MUL'] = df['N_MUL1'] + df['N_MUL2'] + df['N_MUL3']
df['NofCorr ALL DIV'] = df['N_DIV1'] + df['N_DIV2'] + df['N_DIV3']  # Corrected from RT_DIV3 to N_DIV3


In [12]:
df.columns

Index(['age', 'ACC_ADD1', 'ACC_ADD2', 'ACC_ADD3', 'ACC_DIV1', 'ACC_DIV2',
       'ACC_DIV3', 'ACC_MUL1', 'ACC_MUL2', 'ACC_MUL3', 'ACC_SUB1', 'ACC_SUB2',
       'ACC_SUB3', 'RT_ADD1', 'RT_ADD2', 'RT_ADD3', 'RT_DIV1', 'RT_DIV2',
       'RT_DIV3', 'RT_MUL1', 'RT_MUL2', 'RT_MUL3', 'RT_SUB1', 'RT_SUB2',
       'RT_SUB3', 'N_ADD1', 'N_ADD2', 'N_ADD3', 'N_DIV1', 'N_DIV2', 'N_DIV3',
       'N_MUL1', 'N_MUL2', 'N_MUL3', 'N_SUB1', 'N_SUB2', 'N_SUB3',
       'Accuracy ALL level1', 'Accuracy ALL level2', 'Accuracy ALL level3',
       'Accuracy ALL ADD', 'Accuracy ALL SUB', 'Accuracy ALL MUL',
       'Accuracy ALL DIV', 'RT ALL level1', 'RT ALL level2', 'RT ALL level3',
       'RT ALL ADD', 'RT ALL SUB', 'RT ALL MUL', 'RT ALL DIV',
       'NofCorr ALL level1', 'NofCorr ALL level2', 'NofCorr ALL level3',
       'NofCorr ALL ADD', 'NofCorr ALL SUB', 'NofCorr ALL MUL',
       'NofCorr ALL DIV'],
      dtype='object')

In [13]:
def construct_dataset(df, target_column):
    # Determine if the target is aggregated
    is_aggregated_target = 'ALL' in target_column


    
    # Initialize lists to hold columns to include and exclude
    columns_to_exclude = []
    columns_to_include = []
    
    # Base columns categories
    accuracy_columns = [col for col in df.columns if col.startswith('ACC_')]
    reaction_time_columns = [col for col in df.columns if col.startswith('RT_')]
    n_of_corr_columns = [col for col in df.columns if col.startswith('N_') and 'ofCorr' not in col]
    aggregated_columns = [col for col in df.columns if 'ALL' in col]

    relevant = []
    agg_name = ''
    metric = ''
    if ('ACC' in target_column) or ('Accuracy' in target_column):
        metric = 'ACC'
        relevant = accuracy_columns
        agg_name = 'Accuracy'
    elif 'RT' in target_column:
        metric = 'RT'
        relevant = reaction_time_columns
        agg_name = 'RT'
    elif ('N_' in target_column) or ('ofCorr' in target_column):
        metric = 'N' 
        relevant = n_of_corr_columns
        agg_name = 'NofCorr'


    # Exclude components for an aggregated target
    if is_aggregated_target:
        # Identifying the components based on the target
        if 'ADD' in target_column:
            columns_to_exclude.extend([col for col in relevant if 'ADD' in col])
            columns_to_exclude.extend([f'{agg_name} ALL level{level}' for level in range(1,4)])
        elif 'SUB' in target_column:
            columns_to_exclude.extend([col for col in relevant if 'SUB' in col])
            columns_to_exclude.extend([f'{agg_name} ALL level{level}' for level in range(1,4)])

        elif 'MUL' in target_column:
            columns_to_exclude.extend([col for col in relevant if 'MUL' in col])
            columns_to_exclude.extend([f'{agg_name} ALL level{level}' for level in range(1,4)])

        elif 'DIV' in target_column:
            columns_to_exclude.extend([col for col in relevant if 'DIV' in col])
            columns_to_exclude.extend([f'{agg_name} ALL level{level}' for level in range(1,4)])

        elif 'level' in target_column:
            level = target_column.split()[-1][-1]  # Extracting level part
            columns_to_exclude.extend([col for col in relevant if level in col])

            # remove aggregates by operation, otherwise could calculate
            operations = ['ADD', 'SUB', 'MUL', 'DIV']
            columns_to_exclude.extend([f'{agg_name} ALL {operation}' for operation in operations])
    else:
        # For non-aggregated targets, exclude related aggregated columns
        level = target_column.split('_')[-1][-1]
        operation = target_column.split('_')[-1][:-1]
        related_aggregates = [f'{agg_name} ALL {operation}'] + [f'{agg_name} ALL level{level}']
        columns_to_exclude.extend(related_aggregates)

        #remove N with same operation and same level for ACC
        if metric == 'ACC':
            columns_to_exclude.extend([f'N_{operation}{level}'])
    
    # Aggregated columns to always exclude if they are not the target
   # columns_to_exclude.extend([col for col in aggregated_columns if col != target_column])
   # print(columns_to_exclude)
    # Include columns that are not in the exclude list and not the target column
    columns_to_include = [col for col in df.columns if col not in columns_to_exclude and col != target_column]

    # Preparing the dataset
    X = df[columns_to_include]
    y = df[target_column]
    
    return X, y


In [14]:
X, y = construct_dataset(df, 'RT ALL level1')

In [15]:
X.columns

Index(['age', 'ACC_ADD1', 'ACC_ADD2', 'ACC_ADD3', 'ACC_DIV1', 'ACC_DIV2',
       'ACC_DIV3', 'ACC_MUL1', 'ACC_MUL2', 'ACC_MUL3', 'ACC_SUB1', 'ACC_SUB2',
       'ACC_SUB3', 'RT_ADD2', 'RT_ADD3', 'RT_DIV2', 'RT_DIV3', 'RT_MUL2',
       'RT_MUL3', 'RT_SUB2', 'RT_SUB3', 'N_ADD1', 'N_ADD2', 'N_ADD3', 'N_DIV1',
       'N_DIV2', 'N_DIV3', 'N_MUL1', 'N_MUL2', 'N_MUL3', 'N_SUB1', 'N_SUB2',
       'N_SUB3', 'Accuracy ALL level1', 'Accuracy ALL level2',
       'Accuracy ALL level3', 'Accuracy ALL ADD', 'Accuracy ALL SUB',
       'Accuracy ALL MUL', 'Accuracy ALL DIV', 'RT ALL level2',
       'RT ALL level3', 'NofCorr ALL level1', 'NofCorr ALL level2',
       'NofCorr ALL level3', 'NofCorr ALL ADD', 'NofCorr ALL SUB',
       'NofCorr ALL MUL', 'NofCorr ALL DIV'],
      dtype='object')

In [16]:
results = {}

for name in tqdm(df.columns):
    X, y = construct_dataset(df, name)
    
    res = models.LinRegStatmodels(X, y)
    
    r2 = res.rsquared
    params = res.params.index
    
    results[name] = res

100%|██████████| 58/58 [00:21<00:00,  2.75it/s]


In [17]:
results['RT_DIV2'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                RT_DIV2   R-squared (uncentered):                   0.972
Model:                            OLS   Adj. R-squared (uncentered):              0.971
Method:                 Least Squares   F-statistic:                              602.1
Date:                Fri, 22 Mar 2024   Prob (F-statistic):                   6.04e-203
Time:                        16:26:31   Log-Likelihood:                         -520.12
No. Observations:                 290   AIC:                                      1072.
Df Residuals:                     274   BIC:                                      1131.
Df Model:                          16                                                  
Covariance Type:            nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
age                    0.1533      0.038      4.017      0.000       0.078       0.228
ACC_ADD2               3.7185      0.832      4.468      0.000       2.080       5.357
ACC_ADD3              -1.9252      0.555     -3.470      0.001      -3.017      -0.833
ACC_DIV2               4.4423      0.606      7.327      0.000       3.249       5.636
ACC_MUL2              -2.9816      0.745     -4.004      0.000      -4.447      -1.516
RT_ADD2               -0.4769      0.086     -5.567      0.000      -0.646      -0.308
RT_ADD3                0.1886      0.049      3.819      0.000       0.091       0.286
RT_DIV3                0.1543      0.028      5.479      0.000       0.099       0.210
RT_MUL2                0.3634      0.048      7.632      0.000       0.270       0.457
RT_SUB2                0.3627      0.074      4.900      0.000       0.217       0.508
N_DIV1                 0.0895      0.014      6.219      0.000       0.061       0.118
N_DIV2                -0.4304      0.046     -9.354      0.000      -0.521      -0.340
N_DIV3                 0.2391      0.033      7.270      0.000       0.174       0.304
N_MUL2                 0.3777      0.067      5.647      0.000       0.246       0.509
N_SUB2                 0.2884      0.061      4.730      0.000       0.168       0.408
NofCorr ALL level2    -0.1209      0.039     -3.110      0.002      -0.197      -0.044
NofCorr ALL DIV       -0.1018      0.016     -6.465      0.000      -0.133      -0.071
==============================================================================
Omnibus:                       38.106   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               88.194
Skew:                           0.642   Prob(JB):                     7.06e-20
Kurtosis:                       5.377   Cond. No.                     1.13e+16
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is 1.49e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [18]:
params_frequency = {}
for name in df.columns:
    params_frequency[name] = set()

params_frequency['const'] = set()
coefs = {}
for res in results:
    for param_name in results[res].params.index:
        params_frequency[param_name].add(res)
    
    coefs[res] = results[res].params.to_dict()

In [19]:

for name in params_frequency:
    new_input = str(params_frequency[name])
    if new_input == 'set()':
        new_input = ''
    params_frequency[name] = new_input

with open("results/params_in_models_new.json", "w") as outfile:
    json.dump(params_frequency, outfile, indent=4)


r2_dict = {}
for name in results:
    r2_dict[name] = results[name].rsquared

with open("results/r2_LinReg_new.json", "w") as outfile:
    json.dump(r2_dict, outfile, indent=4)


rmse_dict = {}
for name in results:
    rmse_dict[name] = float(np.sqrt(results[name].mse_total))

with open("results/rmse_LinReg_new.json", "w") as outfile:
    json.dump(rmse_dict, outfile, indent=4)


with open("results/coefs_LinReg_new.json", "w") as outfile:
    json.dump(coefs, outfile, indent=4)

In [20]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier



In [21]:
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score


In [22]:
result = {}
for name in tqdm(df.columns):
    X, y = construct_dataset(df, name)

    op, rmse, r2, model = models.XGBReg(X, y)

    result[name] = (op, rmse, r2, model)

100%|██████████| 58/58 [11:55<00:00, 12.34s/it]


In [23]:
result['ACC_ADD1']

((100, 3, 0.5, 0.6),
 0.019328927655170434,
 0.9460695547505464,
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.5, eval_metric=None,
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...))

In [24]:
rmse_dic_xgb = {}
r2_dic_xgb = {}
opt_params = {}
for elem in result:
    r2_dic_xgb[elem] = result[elem][2]
    rmse_dic_xgb[elem] = result[elem][1]
    opt_params[elem] = result[elem][0]
    

In [25]:
with open("results/rmse_XGB_new.json", "w") as outfile:
    json.dump(rmse_dic_xgb, outfile, indent=4)
with open("results/params_XGB_new.json", "w") as outfile:
    json.dump(opt_params, outfile, indent=4)
with open("results/r2_XGB_new.json", "w") as outfile:
    json.dump(r2_dic_xgb, outfile, indent=4)

RF

In [26]:
result = {}
for name in tqdm(df.columns):
    X, y = construct_dataset(df, name)

    op, rmse, r2, model = models.RFReg(X, y)

    result[name] = (op, rmse, r2, model)

  0%|          | 0/58 [00:00<?, ?it/s]

100%|██████████| 58/58 [06:47<00:00,  7.02s/it]


In [27]:
rmse_dic_rf = {}
opt_params = {}
r2_dic_rf = {}

for elem in result:
    r2_dic_rf[elem] = result[elem][2]
    rmse_dic_rf[elem] = result[elem][1]
    opt_params[elem] = result[elem][0]

In [28]:
with open("results/rmse_RF_new.json", "w") as outfile:
    json.dump(rmse_dic_rf, outfile, indent=4)

with open("results/params_RF_new.json", "w") as outfile:
    json.dump(opt_params, outfile, indent=4)

with open("results/r2_RF_new.json", "w") as outfile:
    json.dump(r2_dic_rf, outfile, indent=4)

comparison


In [29]:
with open('results/rmse_RF_new.json') as file:
    res_rf = json.load(file)
with open('results/params_RF_new.json') as file:
    params_rf = json.load(file)

with open('results/rmse_XGB_new.json') as file:
    res_gb = json.load(file)

with open('results/params_XGB_new.json') as file:
    params_gb = json.load(file)

In [30]:
elem

'NofCorr ALL DIV'

In [31]:
FI_RF = {}
for elem in tqdm(res_rf):
    opt_params = params_rf[elem]
    model = RandomForestRegressor(n_estimators=opt_params[0], max_depth=opt_params[1], min_samples_leaf=opt_params[2], min_samples_split=opt_params[3], n_jobs=-1)
    X, y = construct_dataset(df, elem)
    model.fit(X,y)
    FI = list(zip(model.feature_importances_, X.columns))
    FI_RF[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]])


  0%|          | 0/58 [00:00<?, ?it/s]

100%|██████████| 58/58 [01:02<00:00,  1.08s/it]


In [32]:

FI_GB = {}
for elem in tqdm(res_rf):
    opt_params = params_gb[elem]
    model = XGBRegressor(n_estimators = opt_params[0], max_depth=opt_params[1], eta=opt_params[2], colsample_bytree=opt_params[3])

    X, y = construct_dataset(df, elem)
    model.fit(X,y)

    FI = list(zip(model.feature_importances_, model.get_booster().feature_names))
    FI_GB[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]])

# with open('FeatureImportance.json', 'w') as file:
#     json.dump(FI5, file, indent=4)

100%|██████████| 58/58 [00:17<00:00,  3.34it/s]


In [33]:
with open('results/FI_RF.json', 'w') as file:
    json.dump(FI_RF, file, indent=4)

In [34]:
with open('results/FI_GB.json', 'w') as file:
    json.dump(FI_GB, file, indent=4)

In [43]:
opt_models = {}
for elem in tqdm(res_rf):
    diff = res_rf[elem] - res_gb[elem]
    if diff > 0:
        opt_params = params_gb[elem]
        model = XGBRegressor(n_estimators = opt_params[0], max_depth=opt_params[1], eta=opt_params[2], colsample_bytree=opt_params[3])
    else:
        opt_params = params_rf[elem]
        model = RandomForestRegressor(n_estimators=opt_params[0], max_depth=opt_params[1], min_samples_leaf=opt_params[2], min_samples_split=opt_params[3], n_jobs=-1)

    X, y = construct_dataset(df, elem)
    model.fit(X,y)

    opt_models[elem] = model  


FI5 = {}
for elem in opt_models:
    if isinstance(opt_models[elem], XGBRegressor):
        FI = list(zip(opt_models[elem].feature_importances_, opt_models[elem].get_booster().feature_names))
        FI5[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]], 'GB')
    else:
        X, y = construct_dataset(df, elem)
        FI = list(zip(opt_models[elem].feature_importances_, X.columns))
        FI5[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]], 'RF')

    
# with open('FeatureImportance.json', 'w') as file:
#     json.dump(FI5, file, indent=4)

  0%|          | 0/58 [00:00<?, ?it/s]

 50%|█████     | 29/58 [00:16<00:16,  1.81it/s]


KeyboardInterrupt: 

In [41]:
with open('FeatureImportance5_new.json', 'w') as file:
     json.dump(FI5, file, indent=4)